In [1]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas
# !pip install ipywidgets


from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import datadetox as detox
import importlib

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [169]:
pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 7):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

#fitragem série B comjunto 5
df_list[4] = df_list[4][df_list[4]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {  'data': 'Data','Date': 'Data','data_partida': 'Data',
                        'time_mandante': 'Mandante','mandante': 'Mandante','Home': 'Mandante',
                        'time_visitante': 'Visitante','visitante': 'Visitante','Away': 'Visitante'}
    

df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

df_list = detox.padronizar_datas(df_list)

DataFrame 0: Já é datetime64, mantido como está.
DataFrame 1: Convertendo padrão com '/'
DataFrame 2: Convertendo padrão com '/'
DataFrame 3: Convertendo padrão com '/'
DataFrame 4: Convertendo padrão ISO com '-'
DataFrame 5: Já é datetime64, mantido como está.


Unir todos os DFs com agrupamento por colunas, Data, Mandante e Visitante

In [170]:
#mapear cluber qeu o filtro BERT nao resolve (manualmente)
clubes_map_inicial = {  "America-RN":"América Natal","América-RN":"América Natal","América rn":"América Natal",
                        "Joinvile":"Joinville",
                        "Atlético pr":"Athlético-PR","Atlético-PR":"Athlético-PR", 
                        "Grêmio Prudente":"Barueri","Grêmio prudente":"Barueri","Gremio Prudente":"Barueri"}

for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_map_inicial)
    df['Visitante'] = df['Visitante'].replace(clubes_map_inicial)

# listar clubes Mandantes e visitantes, para agrupar ortograficamente
clubes_unicos = []
for df in df_list:
    unicos_por_bd = pd.unique(np.concatenate([df['Mandante'].values, df['Visitante'].values])).tolist()
    clubes_unicos = list(set(clubes_unicos) | set(unicos_por_bd))

clubes_value_dict = [   "América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                        "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                        "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                        "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                        "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                        "Sport Recife", "Vasco da Gama", "Vitória"]

modelo = SentenceTransformer('all-MiniLM-L6-v2')
clubes_corrigidos = detox.corrigir_times_semanticamente(clubes_unicos, clubes_value_dict,modelo)
clubes_dict = detox.gerar_dicionario_mapeamento(clubes_corrigidos, limite=0.0)

# corrigindo e agrupando clubes em todos os dataframes
for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_dict)
    df['Visitante'] = df['Visitante'].replace(clubes_dict)

# unindo todos os dfs com merge on=['Data','Mandante','Visitante']
df_merged = pd.DataFrame(columns=['Data','Mandante','Visitante'])
for i,df in enumerate(df_list):
    df_merged = pd.merge(df_merged, df, how='outer', on=['Data','Mandante','Visitante'],suffixes=('', f'_{i}'))


In [171]:
pd.set_option('display.max_rows', None)
import datetime as dt
df = df_merged.copy()

# Criar edições dos campeonatos visto que 2020 foi ate 2021
mask = (df['Data'] > dt.datetime(2020, 8, 1)) & (df['Data'] < dt.datetime(2021, 2, 28))
df.loc[mask, 'edition'] = 2020
df.loc[~mask, 'edition'] = df.loc[~mask, 'Data'].dt.year

df.sort_values(by=['Mandante', 'Visitante','edition','Data'], ascending=[True, True, True, True], inplace=True)
#df.head(10)

In [174]:
importlib.reload(detox)

# Agrupar e aplicar a função
df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)

# Aplica o agrupamento e mescla as linhas sem conflito
df_mesclado = (
    df_ajustado.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado = df_mesclado[df_mesclado['edition']<2024]

df_group = df_mesclado.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(
    lambda group: detox.ajustar_datas_semdiff(group,keep=True))

df_mesclado2 = (
    df_group.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado2.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx',index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_13184\2427877954.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)
C:\Users\USER\AppData\Local\Temp\ipykernel_13184\2427877954.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(detox.mesclar_linhas_sem_conflito)
C:\Users\USER\AppData\Local\Temp\ipykernel_13184\2427877954.py:9: DeprecationWarn

Garantindo partidas por temporada

In [272]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
importlib.reload(detox)

df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx')

#ajustando sao paulo e sao caetano jogo interrompido
df.drop(1007, inplace=True)

# ajustando jogo do flamengo
df.loc[2835, 'Data'] = pd.to_datetime('2009-11-07')
df.drop(2834, inplace=True)

# ajustando jogo do BOTAFOGO
df.loc[1969, 'Data'] = pd.to_datetime('2009-07-29')
df.drop(1968, inplace=True)

# ajustando jogo da chapecoense
df.drop(5753, inplace=True)

#2005
df.drop([1137,1137,1207,1231,1257,1284,1307,1310,1328,1341,1375,1379], inplace=True)
df.loc[1138, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1208, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1232, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1258, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1285, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1308, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1311, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1329, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1342, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1376, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1380, 'Data'] = pd.to_datetime('2009-10-19')

#2007 
df.drop([1970,1971], inplace=True)


detox.verifica_partidas_edi(df,'edition','Mandante')
detox.verifica_partidas_edi(df,'edition','Visitante')

df.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx',index=False)

Edições com número desigual de partidas por time: edition
2016    2
Name: n_partidas, dtype: int64
Edições com número desigual de partidas por time: edition
2016    2
Name: n_partidas, dtype: int64


Meslclando Colunas

In [14]:
df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx')

gols_mandante = ["gols_mandante","mandante_Placar","Mandante Placar","mandante_Placar_3","Gol Mandante","placar_mandante"]
gols_visitante = ["gols_visitante","visitante_Placar","Visitante Placar","visitante_Placar_3","Gol Visitante","placar_visitante"]
colunas_hora = ["Horário", "hora", "hora_3", "Hora", "hora_partida"]

for col in colunas_hora:
    df[col] = df[col].str.replace(r'h', ':', regex=True)
    #display(df[col].str.match(r'^\d{2}:\d{2}$').value_counts())
    df[col] = pd.to_datetime(df[col], format='%H:%M')

df[gols_mandante].nunique(axis=1,dropna=True).value_counts()
df[gols_visitante].nunique(axis=1,dropna=True).value_counts()
df[colunas_hora].nunique(axis=1,dropna=True).value_counts()

df['gols_mandante_moda'] = df[gols_mandante].mode(axis=1).iloc[:, 0]
df['gols_visitante_moda'] = df[gols_visitante].mode(axis=1).iloc[:, 0]
df['horario'] = df[colunas_hora].mode(axis=1).iloc[:, 0]
df.drop(columns=gols_mandante+gols_visitante+colunas_hora, inplace=True)

In [15]:
importlib.reload(detox)

df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=1,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=2,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=3,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=4,div=15))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=5,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=6,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=7,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=8,div=30))
df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=9,div=30))
df.loc[2191, 'horario'] = pd.to_datetime('2009-10-19 19:00') #horario meio noite
df['horario'] = df['horario'].dt.time

In [22]:
importlib.reload(detox)
arenas_col = ['arena', 'Arena', 'arena_3', 'Estádio', 'estadio']

# df[gols_mandante].nunique(axis=1,dropna=True).value_counts()
# df['Arenas'] = df[arenas_col].mode(axis=1).iloc[:, 0]

for col in arenas_col:
    df[col] = df[col].apply(detox.normalizar_texto)
    df[col] = df[col].str.replace(r'\bestadio\b', '', regex=True).str.strip()
    df[col] = df[col].str.replace(r'\(\*pf\)', '', regex=True).str.strip()
    df[col] = df[col].str.replace(r'\*\(pf\)', '', regex=True)






In [27]:
#lista fornecida pelo chat gpt
mapeamento_estadios = {
    "olimpico engenhao": "nilton santos",
    "engenhao": "nilton santos",
    "arena corinthians": "neo quimica arena",
    "neo quimica arena": "neo quimica arena",
    "municipal paulo machado de carvalho": "pacaembu",
    "urbano caldeira": "vila belmiro",
    "jose pinheiro borda": "beira-rio",
    "beira rio": "beira-rio",
    "arena castelao": "castelao",
    "placido castelo": "castelao",
    "castelao de sao luis": "castelao",
    "parque antartica": "allianz parque",
    "palestra italia": "allianz parque",
    "itaipava arena fonte nova": "fonte nova",
    "arena fonte nova": "fonte nova",
    "da serrinha": "serrinha",
    "adelmar da costa carvalho": "ilha do retiro",
    "raimundo sampaio": "independencia",
    "do governo do estado de goias (serra dourada)": "serra dourada",
    "nabizao": "nabi abi chedid",
    "kyocera arena": "ligga arena",
    "arena da baixada": "ligga arena",
    "joaquim americo guimaraes": "ligga arena",
    "municipal general raulino de oliveira": "raulino de oliveira",
    "haile pinheiro - serrinha" : "serrinha",
    "do vale": "estadio do vale",
    "claudio moacyr": "claudio moacyr de azevedo",
    "municipal parque do sabia": "parque do sabia",
    "moacyrzao": "claudio moacyr de azevedo",
    "do cafe": "estadio do cafe",
    "alberto oliveira": "joia da princesa",
    "vasco da gama": "sao januario",
    "doutor adhemar de barros": "fonte luminosa",
    "santa cruz": "arruda",
    "benedito teixeira": "teixeirao",
    "melao": "estadio do melao",
    "pres vargas": "presidente vargas",
    "mario helenio": "helenao",
    "batistao": "arena batistao",
    "romildao": "vail chaves",
    "wilson de barros": "vail chaves",
    "eduardo jose farah": "prudentao",
    "g coutinho": "giulite coutinho",
    "bruno j daniel ": "bruno jose daniel",
    "mj jose levi sobrinho": "limeirao",
    "pedro pedrossian": "morenao",
    "vivaldo lima": "arena da amazonia",
    "edson passos": "giulite coutinho",
    "municipal juiz de fora": "helenao",
    "a campanella": "anacleto campanella",
    "bruno j.daniel": "bruno jose daniel",
    "bruno j daniel": "bruno jose daniel",
    "luiz lacerda": "lacerdao",
    "r de oliveira": "raulino de oliveira",
    "serejao": "boca do jacare",
    "a.campanella": "anacleto campanella",
    "papa joao paulo ii": "vail chaves",
    "papa j.paulo ii": "vail chaves",
    "municipal juscelino kubitschek": "juscelino kubitscheck",
    "juiz de fora":"helenao",
    "romildo ferreira": "vail chaves",
    "paulo constantino": "prudentao",
    "joaquim henrique nogueira-arena do jacare": "arena do jacare",
    "luso-brasileiro": "luso brasileiro",
    "arena brb mane garrincha": "mane garrincha",
    "bruno josé daniel": "bruno jose daniel",
    "de pituacu": "pituacu",
    "texeirao": "teixeirao",
    "arena de pernambuco": "arena pernambuco",
    "arena da amazonia":"arena amazonia",
    "nacional de brasilia": "mane garrincha",
    "durival de brito": "vila capanema",}





for arena in arenas_col:
    df[arena] = df[arena].replace(mapeamento_estadios)

df['Arenas_iguais'] = df[arenas_col].nunique(axis=1, dropna=True) == 1
#display(df[arenas_col+['Arenas_iguais']])

# indicando a moda maior que 2
df['Arena_moda'] = df[arenas_col].mode(axis=1, dropna=True).iloc[:, 0]
#df['Arena_moda'] = df['Arena_moda'].apply(lambda x: x if x > 2 else None)

mask = df['Arenas_iguais'] == False
#display(df[mask][arenas_col])

pd.set_option('display.max_rows', None)
display(df['Arena_moda'].value_counts())

Arena_moda
maracana                            630
mineirao                            459
morumbi                             413
beira-rio                           369
ligga arena                         359
vila belmiro                        357
nilton santos                       299
couto pereira                       297
serra dourada                       295
pacaembu                            278
independencia                       274
allianz parque                      262
sao januario                        261
castelao                            251
orlando scarpelli                   219
neo quimica arena                   185
barradao                            183
arena do gremio                     183
ilha do retiro                      181
moises lucarelli                    179
olimpico                            172
fonte nova                          155
alfredo jaconi                      146
ressacada                           134
arena conda                  